<a href="https://colab.research.google.com/github/ikfaisal/dsia/blob/master/DSIA_Lab_2_1_3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.3 *Database Programming with Python and SQLite*

<a name="demo"></a>
# Using SQLite

## Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



## The `sqlite3` Command Line Utility

- useful for basic SQL tasks and databse maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands". 

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


## The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [1]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [2]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
conn.commit()


OperationalError: table houses already exists

In [8]:
c.fetchall()

[]

In [3]:
c.execute('DROP TABLE houses;')

# Save (commit) the changes:
conn.commit()

In [4]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
conn.commit()

With the database saved the table should now be viewable using SQLite Manager.

## Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [5]:
# A
last_sale = (None, 4000, 5, 22, 619000)

# Execute Command
c.execute("insert into houses values (?, ?, ?, ?, ?);", last_sale )

# Remember to commit the changes
conn.commit()

In [6]:
c.execute("SELECT * FROM houses")
print(c.fetchone())

(1, 4000, 5, 22, 619000)


Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key. 

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> c.executemany(sql_command, values)

In [8]:
# A
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
# this is a security measure against SQL injection attacks:
# 
c.executemany("insert into houses values (?, ?, ?, ?, ?)", recent_sales )

conn.commit()

Select all rows from houses

In [9]:
# A
c.execute("SELECT * FROM houses")


In [10]:
for row in c.execute('SELECT * FROM houses'):
        print(row)

(1, 4000, 5, 22, 619000)
(2, 2390, 4, 34, 319000)
(3, 1870, 3, 14, 289000)
(4, 1505, 3, 90, 269000)


## Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more. 

In [11]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:
data = genfromtxt('C:/Users/user/Documents/IOD_data_MAIN/DATA/housing-data.csv', dtype = 'i8', 
                    delimiter = ',', skip_header = 1).tolist()

Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [12]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None)

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [13]:
# check:
print(type(data))
print(data[0:3])

<class 'list'>
[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000]]


Now we can insert each list item as a row of fields in the database.

In [14]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In [22]:
for row in c.execute('SELECT * FROM houses'):
        print(row[:])

(1, 4000, 5, 22, 619000)
(2, 2390, 4, 34, 319000)
(3, 1870, 3, 14, 289000)
(4, 1505, 3, 90, 269000)
(5, 2104, 3, 70, 399900)
(6, 1600, 3, 28, 329900)
(7, 2400, 3, 44, 369000)
(8, 1416, 2, 49, 232000)
(9, 3000, 4, 75, 539900)
(10, 1985, 4, 61, 299900)
(11, 1534, 3, 12, 314900)
(12, 1427, 3, 57, 198999)
(13, 1380, 3, 14, 212000)
(14, 1494, 3, 15, 242500)
(15, 1940, 4, 7, 239999)
(16, 2000, 3, 27, 347000)
(17, 1890, 3, 45, 329999)
(18, 4478, 5, 49, 699900)
(19, 1268, 3, 58, 259900)
(20, 2300, 4, 77, 449900)
(21, 1320, 2, 62, 299900)
(22, 1236, 3, 78, 199900)
(23, 2609, 4, 5, 499998)
(24, 3031, 4, 21, 599000)
(25, 1767, 3, 44, 252900)
(26, 1888, 2, 79, 255000)
(27, 1604, 3, 13, 242900)
(28, 1962, 4, 53, 259900)
(29, 3890, 3, 36, 573900)
(30, 1100, 3, 60, 249900)
(31, 1458, 3, 29, 464500)
(32, 2526, 3, 13, 469000)
(33, 2200, 3, 28, 475000)
(34, 2637, 3, 25, 299900)
(35, 1839, 2, 40, 349900)
(36, 1000, 1, 5, 169900)
(37, 2040, 4, 75, 314900)
(38, 3137, 3, 67, 579900)
(39, 1811, 4, 24, 285900

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [23]:
import numpy as np
d1 = np.asarray([1200, 3, 15, 250000])
d1 = d1.tolist()
d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()

## Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [24]:
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)')
conn.commit()

In [25]:
for row in c.execute('SELECT * FROM houses'):
        print(row[:])

(1, 4000, 5, 22, 619000)
(2, 2390, 4, 34, 319000)
(3, 1870, 3, 14, 289000)
(4, 1505, 3, 90, 269000)
(5, 2104, 3, 70, 399900)
(6, 1600, 3, 28, 329900)
(7, 2400, 3, 44, 369000)
(8, 1416, 2, 49, 232000)
(9, 3000, 4, 75, 539900)
(10, 1985, 4, 61, 299900)
(11, 1534, 3, 12, 314900)
(12, 1427, 3, 57, 198999)
(13, 1380, 3, 14, 212000)
(14, 1494, 3, 15, 242500)
(15, 1940, 4, 7, 239999)
(16, 2000, 3, 27, 347000)
(17, 1890, 3, 45, 329999)
(18, 4478, 5, 49, 699900)
(19, 1268, 3, 58, 259900)
(20, 2300, 4, 77, 449900)
(21, 1320, 2, 62, 299900)
(22, 1236, 3, 78, 199900)
(23, 2609, 4, 5, 499998)
(24, 3031, 4, 21, 599000)
(25, 1767, 3, 44, 252900)
(26, 1888, 2, 79, 255000)
(27, 1604, 3, 13, 242900)
(28, 1962, 4, 53, 259900)
(29, 3890, 3, 36, 573900)
(30, 1100, 3, 60, 249900)
(31, 1458, 3, 29, 464500)
(32, 2526, 3, 13, 469000)
(33, 2200, 3, 28, 475000)
(34, 2637, 3, 25, 299900)
(35, 1839, 2, 40, 349900)
(36, 1000, 1, 5, 169900)
(37, 2040, 4, 75, 314900)
(38, 3137, 3, 67, 579900)
(39, 1811, 4, 24, 285900

QUIZ: What would `DELETE FROM houses` do?

## Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [76]:
# ANSWER bdrms
for row in c.execute('SELECT * FROM houses WHERE bdrms = 4'):
        print(row[:])

(2, 2390, 4, 34, 319000)
(9, 3000, 4, 75, 539900)
(10, 1985, 4, 61, 299900)
(15, 1940, 4, 7, 239999)
(20, 2300, 4, 77, 449900)
(23, 2609, 4, 5, 499998)
(24, 3031, 4, 21, 599000)
(28, 1962, 4, 53, 259900)
(37, 2040, 4, 75, 314900)
(39, 1811, 4, 24, 285900)
(42, 2132, 4, 28, 345000)
(43, 4215, 4, 66, 549000)
(44, 2162, 4, 43, 287000)
(47, 2567, 4, 57, 314000)
(50, 1852, 4, 64, 299900)
(56, 3000, 4, 75, 539900)
(57, 1985, 4, 61, 299900)
(62, 1940, 4, 7, 239999)
(67, 2300, 4, 77, 449900)
(70, 2609, 4, 5, 499998)
(71, 3031, 4, 21, 599000)
(75, 1962, 4, 53, 259900)
(84, 2040, 4, 75, 314900)
(86, 1811, 4, 24, 285900)
(89, 2132, 4, 28, 345000)
(90, 4215, 4, 66, 549000)
(91, 2162, 4, 43, 287000)
(94, 2567, 4, 57, 314000)
(97, 1852, 4, 64, 299900)


**2. Run a query to calculate the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [99]:
# ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()


# Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [3]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

## Writing data into a database

Data in Pandas can be loaded into a relational database. 

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [4]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv('C:/Users/user/Documents/IOD_data_MAIN/DATA/housing-data.csv', low_memory = False)
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [118]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)

In [120]:

query = """
        SELECT * 
        FROM houses_pandas 
        LIMIT 10;
        """

orders_head_10 = sql.read_sql(query, con= conn)
orders_head_10

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900
5,1985,4,61,299900
6,1534,3,12,314900
7,1427,3,57,198999
8,1380,3,14,212000
9,1494,3,15,242500


Run a query to get the average price of each house size from this table:

In [122]:
#ANSWER
houseprice_avg = c.execute("SELECT price, AVG(price) AS AVG_price FROM houses_pandas)
houseprice_avg.fetchall()

SyntaxError: EOL while scanning string literal (<ipython-input-122-420a6666769a>, line 2)

## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database?

2. When would you want to use Pandas on a dataset retrieved from a database?

# Reference: SQL Syntax 


## SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

## WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

## AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate funtions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

# Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

## Read CSV into DataFrame

In [5]:
import pandas as pd
import sqlite3
from pandas.io import sql

In [7]:
# Reading CSV to Dataframe
orders = pd.read_csv('C:/Users/user/Documents/IOD_data_MAIN/DATA/P12-ListOfOrders.csv', encoding = 'utf-8')
orders_break_down =  pd.read_csv('C:/Users/user/Documents/IOD_data_MAIN/DATA/P12-OrderBreakdown.csv', encoding = 'utf-8')

In [9]:
orders.head()

,Order ID,Order Date,Customer Name,Country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [10]:
orders_break_down.head()

,Order ID,Product Name,Discount,Sales,Quantity,Category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


## Replace Space with Underscore in Column Names

In [11]:
# ANSWER
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]

In [12]:
# ANSWER
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]

## Check DataTypes 

In [13]:
# ANSWER
orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [14]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
quantity          int64
category         object
dtype: object

## Save these two dataframes as a table in sqlite

In [15]:
# Establishing Local DB connection
db_connection = sqlite3.connect('C:/Users/user/Documents/IOD_data_MAIN/DATA/eshop.db.sqlite')

# Save these two dataframes as a table in sqlite
orders.to_sql('orders_pandas', con = db_connection, if_exists = 'replace', index = False)
orders_break_down.to_sql('orders_break_down', con = db_connection, if_exists = 'replace', index = False)

## Select Number of Orders for Each Customer

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [17]:
# Select first 10 rows of all columns in orders table

query = """
        SELECT * 
        FROM orders_pandas 
        LIMIT 20;
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France
5,ES-2011-5433855,2011-01-07,Dave Poirier,France
6,IT-2011-4546695,2011-01-08,Darren Powers,France
7,ES-2011-1138719,2011-01-11,Eric Murdock,Italy
8,ES-2011-1466305,2011-01-11,Mick Brown,Austria
9,ES-2011-4359424,2011-01-11,Dorothy Dickinson,Spain


In [29]:
# Select first 10 rows of all columns in orders table

query = """
        SELECT * 
        FROM orders_break_down 
        LIMIT 10;
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,order_id,product_name,discount,sales,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies
5,ES-2011-3848439,"Binney & Smith Pencil Sharpener, Water Color",0.0,90.0,3,Office Supplies
6,ES-2011-3848439,"Sanford Canvas, Fluorescent",0.0,207.0,4,Office Supplies
7,ES-2011-5433855,"Bush Floating Shelf Set, Pine",0.1,155.0,1,Furniture
8,ES-2011-5433855,"Accos Thumb Tacks, Assorted Sizes",0.0,33.0,3,Office Supplies
9,ES-2011-5433855,"Smead Lockers, Industrial",0.1,716.0,4,Office Supplies


In [18]:
# Select Number of Orders for Each Customer


query = """
       SELECT customer_name, 
        COUNT(order_id) as number_of_order 
        from orders_pandas
        GROUP BY (customer_name)
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,customer_name,number_of_order
0,Aaron Bergman,11
1,Aaron Hawkins,7
2,Aaron Smayling,12
3,Adam Bellavance,4
4,Adam Hart,8
...,...,...
787,Xylona Preis,2
788,Yana Sorensen,5
789,Yoseph Carroll,11
790,Zuschuss Carroll,9


## Select Number of Customers for Each Country

In [21]:
# Select Number of Customers for Each Country


query = """
       SELECT country,
        COUNT(customer_name) AS Number_of_cust_country
from orders_pandas
GROUP BY country
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,country,Number_of_cust_country
0,Austria,135
1,Belgium,68
2,Denmark,29
3,Finland,34
4,France,991
5,Germany,806
6,Ireland,50
7,Italy,493
8,Netherlands,194
9,Norway,37


## Select discount, sales, quantity for Each Order from orders_break_down Table

In [43]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table


query = """
       SELECT SUM(discount) AS total_discount
       from orders_break_down
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,total_discount
0,885.55


In [34]:

query = """
       SELECT SUM( sales) AS total_sales
       from orders_break_down
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,total_sales
0,2348482.0


In [26]:


query = """
       SELECT quantity from orders_break_down
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,quantity
0,3
1,7
2,3
3,2
4,2
...,...
8042,2
8043,2
8044,4
8045,3


##  Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [38]:
query = """
       SELECT *
       from orders_break_down
       limit 10
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,order_id,product_name,discount,sales,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies
5,ES-2011-3848439,"Binney & Smith Pencil Sharpener, Water Color",0.0,90.0,3,Office Supplies
6,ES-2011-3848439,"Sanford Canvas, Fluorescent",0.0,207.0,4,Office Supplies
7,ES-2011-5433855,"Bush Floating Shelf Set, Pine",0.1,155.0,1,Furniture
8,ES-2011-5433855,"Accos Thumb Tacks, Assorted Sizes",0.0,33.0,3,Office Supplies
9,ES-2011-5433855,"Smead Lockers, Industrial",0.1,716.0,4,Office Supplies


In [35]:
# Select discount, sales, quantity, total price for Each Order from orders_break_down Table


query = """
       SELECT sum(quantity) as total_quantity
       from orders_break_down
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,total_quantity
0,30354


In [39]:


query = """
       SELECT sum(sales) * sum(quantity) - sum(discount) as total_price 
       from orders_break_down
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,total_price
0,7.128582e+10


In [44]:
query = """
       SELECT sales * quantity - discount as total_price 
       SELECT sales * quantity * (1- discount) as percentage_total_price
       from orders_break_down
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

DatabaseError: Execution failed on sql '
       SELECT sales * quantity - discount as total_price 
       SELECT sales * quantity * (1- discount) as percentage_total_price
       from orders_break_down
        ': near "SELECT": syntax error

In [42]:
query = """
       SELECT *
       from orders_break_down
       limit 10
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,order_id,product_name,discount,sales,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies
5,ES-2011-3848439,"Binney & Smith Pencil Sharpener, Water Color",0.0,90.0,3,Office Supplies
6,ES-2011-3848439,"Sanford Canvas, Fluorescent",0.0,207.0,4,Office Supplies
7,ES-2011-5433855,"Bush Floating Shelf Set, Pine",0.1,155.0,1,Furniture
8,ES-2011-5433855,"Accos Thumb Tacks, Assorted Sizes",0.0,33.0,3,Office Supplies
9,ES-2011-5433855,"Smead Lockers, Industrial",0.1,716.0,4,Office Supplies


## Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [40]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100


query = """
      SELECT *
from orders_break_down
WHERE total_price > 100
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

DatabaseError: Execution failed on sql '
      SELECT *
from orders_break_down
WHERE total_price > 100
        ': no such column: total_price

## Select All Customer And The Product They Have Bought

In [0]:
# Select All Customer And The Product They Have Bought


## Select Number of 'Furniture' Orders For Each Country 

In [0]:
# Select Number of 'Furniture' Orders For Each Country

## Select Number of 'Furniture' Orders For The Country Denmark

In [0]:
# Select Number of 'Furniture' Orders For The Country Denmark

## Select Total Sales With Discount and Without Discount for Each Country

In [0]:
# Select Total Sales With Discount and Without Discount for Each Country

## Select Total Quantity, Total Sales for Each Country

In [0]:
# Select Total Quantity, Total Sales for Each Country

## Select Top 3 Country Based on Sales

In [0]:
# Select Top 3 Country Based on Sales

## Select Bottom 3 Country Based On Quantities

In [0]:
# Select Bottom 3 Country Based On Quantities

## Select Average Sales By Categroy For The Country 'France'

In [0]:
# Select Average Sales By Categroy For The Country 'France'

## Select Country, Category and Total Sales Where Average Total Sales is The Highest

In [0]:
# Select Country, Category and Total Sales Where Average Total Sales is The Highest

# JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams. 
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

#  ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
- [SQL zoo](http://www.sqlzoo.net)  Great for learning syntax